In [44]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import pickle
import time as time
import math, cmath
import sys
#import proj_ev_library as projev
#import max_entev library as meev

from IPython.display import display, Math, Latex

In [47]:
#import simpy no hace falta

total_no_spins = 50 ### este es el nÃºmero de spines en el lattice
coherences_t0_pert0 = 1. ### el coeficiente I_m^0(t) inicial, son las coherencias a tiempo 0 sin perturbaciÃ³n. Modificar dsps.
coeff_list_t0 = [coherences_t0_pert0 + np.random.rand() for i in range(total_no_spins)] ### una configuraciÃ³n inicial 
p = .5 ### el tÃ©rmino que pondera el avg del hamiltoniano del estado inicial con la perturbacion
#cmts_at_t0 = [1] + [np.random.rand() for i in range(49)]

### Este mÃ³dulo construye la matrix que relaciona los coeficientes c_m(t). Es re contra fea pero it gets the job done

def construct_M_matrix(coeff_list):
    m_matrix_list = []
    ith_cmt_list = []
    
    ### La ec de markov la podemos escribir como d/dt(C(t)) = -i/I_M^0(t) * M (C(t))
    
    ### la macana es que, con condiciones de borde libres, tengo que cargar a mano los dos primeros y dos Ãºltimos cm(t)s
    ### del lattice. Capaz con un condiciones periÃ³dicas, pensÃ¡ndo la lattice como un Toro T_1, queda mÃ¡s lindo.
    
    first_cmt_list = [(coeff_list[0+2]-coeff_list[0]) * np.e**(-0) - (coeff_list[0]) * np.e**(-0)
                       + p * np.e**(-0)]
    m_matrix_list.append(first_cmt_list)
    
    second_cmt_list = [(coeff_list[1+2]-coeff_list[1]) * np.e**(-1) - (coeff_list[1]) * np.e**(-1)
                       + p * np.e**(-1)]    
    m_matrix_list.append(first_cmt_list)
    
    for i in range(2, len(coeff_list) - 2):
        ith_cmt_list = [(coeff_list[i+2]-coeff_list[i]) * np.e**(-m) + (coeff_list[i-2]-coeff_list[i]) * np.e**(-m)
                       + p * np.e**(-m)]
        m_matrix_list.append(ith_cmt_list)
        ith_cmt_list = []
    
    penultimate_cmt_list = [(coeff_list[M-2+2-1]-coeff_list[M-2-1]) * np.e**(-(M-2-1)) 
                            - (coeff_list[M-2-2-1]-coeff_list[M-2-1]) * np.e**(-(M-2-1))
                       + p * np.e**(-(M-2-1))]
    m_matrix_list.append(penultimate_cmt_list)
    
    ultimate_cmt_list = [-(coeff_list[M-1]) * np.e**(-(M-1)) 
                            - (coeff_list[M-1-2]-coeff_list[M-1]) * np.e**(-(M-1))
                       + p * np.e**(-(M-1))]
    m_matrix_list.append(ultimate_cmt_list)
    m_matrix = np.array(m_matrix_list)
    
    return m_matrix#, m_matrix_list

Mij_matrix = construct_M_matrix(coeff_list)

In [48]:
Mij_matrix ### revisar porque no es una matriz

array([[-7.11189088e-01],
       [-7.11189088e-01],
       [-7.50340629e-02],
       [ 2.95456526e-02],
       [ 1.64887684e-01],
       [ 1.44649709e-01],
       [ 1.45767076e-01],
       [ 2.52969806e-02],
       [-5.55359248e-02],
       [ 1.71446892e-01],
       [ 1.78608150e-01],
       [-1.16885921e-01],
       [-1.16188860e-02],
       [ 1.41722745e-01],
       [-2.96633372e-02],
       [ 1.66966982e-01],
       [ 1.91483629e-01],
       [-8.17905483e-02],
       [ 1.02465796e-01],
       [ 1.35069893e-01],
       [ 2.88798129e-02],
       [ 1.03528912e-01],
       [ 1.05409905e-01],
       [ 3.61449435e-02],
       [-8.72903380e-04],
       [ 4.13558915e-02],
       [ 9.33766910e-02],
       [ 1.56989196e-01],
       [-1.98500860e-02],
       [-9.43796966e-02],
       [ 1.98001852e-01],
       [ 1.90254579e-01],
       [ 4.02359255e-02],
       [ 5.96460025e-02],
       [ 4.52730625e-03],
       [ 4.50084062e-02],
       [ 9.94160392e-02],
       [ 4.56937859e-03],
       [ 1.6

In [46]:
from scipy.integrate import odeint

def F(y, t, H_tensor, coherences_t0_pert0):
    return H_tensor.dot(y)/(coherences_t0_pert0) # * (-1j) primero busco de resolver una ec real que es mÃ¡s facil xd

ts=np.linspace(0,1,200)
result = odeint(func=F, y0=coeff_list_t0,t=ts,args=(Mij_matrix, coherences_t0_pert0, ))
plt.figure(figsize=(4, 3))

for i in [0, 5, 10, 15, 49]:
    plt.plot(result[i], label=r'$\phi_{(%d)}(t)$'%i)

plt.plot(result)
plt.xlabel("t")
plt.ylabel("Phi(t)")
plt.legend()

ValueError: shapes (50,1) and (50,) not aligned: 1 (dim 1) != 50 (dim 0)